In [4]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from netgen.csg import unit_cube
from ngsolve.webgui import Draw
from netgen.occ import *


In [5]:
box = Box((0,0,0), (3,0.6,1))
box.faces.name="outer"
cyl = sum( [Cylinder((0.5+i,0,0.5), Y, 0.25,0.8) for i in range(3)] )
cyl.faces.name="cyl"
geo = box-cyl
cylboxedges = geo.faces["outer"].edges * geo.faces["cyl"].edges
cylboxedges.name = "cylbox"
geo = geo.MakeChamfer(cylboxedges, 0.03)
geo.faces.Min(X).name = "fix"
geo.faces.Max(X).name = "force"
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=2)).Curve(3)
Draw (mesh);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [9]:
matL2 = MatrixValued(L2(mesh,order=2), dim=3, symmetric=True)

H1fes = VectorH1(mesh, order=2, dirichlet="fix")

X = FESpace([matL2, H1fes,matL2])
eps, u, sigma = X.TrialFunction()
eta,v,tau = X.TestFunction()



E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))
force = CF( (1e-3,0,0) )
def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)

bf = BilinearForm(X)
bf += InnerProduct(eps, Stress(eta)) * dx - InnerProduct(eta, sigma) * dx

bf+= InnerProduct(Grad(v), sigma) * dx 
bf+= InnerProduct(Grad(u), tau) * dx - InnerProduct(tau, eps) * dx
bf.Assemble()
l = LinearForm(X)
l += InnerProduct(force, v) * dx
l.Assemble()
sol = GridFunction(X)
print("Inverting...")
sol.vec.data = bf.mat.Inverse(X.FreeDofs()) * l.vec
e,gfu,s = sol.components
Draw (Norm(s), mesh, "u", deformation=True)


Inverting...


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [7]:
Draw (gfu.components[0], mesh, "u", deformation=True)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene